# Get data and create input files for BASE-9

This notebook tests my codes to grab data from Gaia, Pan-STARRS, and 2MASS, then use Gaia data to estimate membership probabilities, and output a phot file for BASE-9

## Photometry will be downloaded from

### Gaia:
- https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html
- https://gea.esac.esa.int/archive/documentation/GDR3/index.html
- https://gaia.aip.de/metadata/gaiadr3/

### Pan-STARRS:
- https://gaia.aip.de/metadata/gaiaedr3/panstarrs1_best_neighbour/
- https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_external_catalogues/ssec_dm_panstarrs1_original_valid.html
- https://allendowney.github.io/AstronomicalData/05_join.html

### 2MASS:
- https://www.cosmos.esa.int/web/gaia-users/archive/writing-queries
- https://gaia.aip.de/metadata/gaiaedr3/tmass_psc_xsc_best_neighbour/
- https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_crossmatches/ssec_dm_tmass_psc_xsc_join.html
- https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_external_catalogues/ssec_dm_tmass_original_valid.html



In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# if this is your first time running, you may need to create a config file for the dustmap code

# from dustmaps.config import config
# config.reset()

In [ ]:
# our code

%load_ext autoreload
%autoreload 2
from getGaiaData import GaiaClusterMembers


## 1. Set the initial parameters for the cluster

In [ ]:
# define the cluster worker 
cluster = GaiaClusterMembers()

In [ ]:
clusterName ='NGC_2682' #Define cluster here, named as in the `OCcompiled_clean_v2.csv` file
pwd = clusterName +'_dir' #Path to directory where output to be stored
modelDirectory = '/projects/p31721/BASE9/base-models/' #Path where the stellar evollution models for BASE-9 are stored

In [ ]:
# typically values in this cell do not need to be edited

OCdf = pd.read_csv('OCcompiled_clean_v2.csv') #file with prior values for cluster centers from previous literature
row = OCdf.loc[OCdf['ID'] == clusterName].iloc[0]
dmod = 5.*np.log10(row['dist[pc]']) - 5.
err_dmod = (5.*1./np.log(10.)*(1./row['dist[pc]']))*row['err_dist[pc]'] # error propagation
try:
    os.makedirs(pwd)
    os.makedirs(os.path.join(pwd,'plots'))
except:
    print('WARNING: unable to create directory:',pwd)
    next

cluster.verbose = 1
cluster.plotNameRoot = os.path.join(pwd,'plots')
cluster.photOutputFileName = os.path.join(pwd,clusterName + '.phot')
cluster.saveDataFile = os.path.join(pwd, clusterName + '.ecsv')
cluster.yamlOutputFileName = os.path.join(pwd,'base9.yaml')
cluster.yamlTemplateFileName = 'template_base9.yaml'
cluster.yamlInputDict = {
    'photFile' : clusterName + '.phot',
    'outputFileBase' : pwd,
    'modelDirectory' : modelDirectory,
    'msRgbModel' : 5,
    'Fe_H' : [row['[Fe/H]'], row['[Fe/H]'], np.max([row['err_[Fe/H]'], 0.3])],
    'Av' : [row['Av[mag]'], row['Av[mag]'], np.max([row['err_Av[mag]'], 0.3])],
    'Y' : [0.29, 0.29, 0.0], # keep the defaults
    'carbonicity' : [0.38, 0.38, 0.0], # keep the defaults
    'logAge' : [np.log10(row['age[Myr]']*10**6), np.log10(row['age[Myr]']*10**6), np.inf],
    'distMod' : [dmod, dmod, np.max([err_dmod, 1.0])],
}

cluster.sig_fac = 10 #default is to include stars within 10 sigma bounds around cluster fit mean.
cluster.deredden = True #Set to True to apply reddening corrections.



## 2. Run the code to download data and generate files

In [ ]:
# cluster.runAll(clusterName) #run this line to download Gaia data that will be saved to the file clusterName+'_dir/'+clusterName+'_GaiaData.ecsv'
cluster.runAll(clusterName, filename=cluster.saveDataFile) #run this line if Gaia data has already been downloaded


## 3. (Optional) Run interactive tool to remove field star manually

For rich clusters, sometimes the Gaia memberships do not sufficiently remove field stars (e.g., stars far from the isochrone).  The tool below allows users to interactively select obvious field stars for removal.  The tool will also allow users to modify the starting values of the isochrone model by visually adjusting the isochrone fit.

*Note:* you can define the filters to use in the CMD plots by setting keys in the cluster object (see `getGaiaData.py`).  Here is a list of the filters you can input for the mag, color1, and color2 arguments below.

 - Gaia filters: `phot_g_mean_mag`, `phot_bp_mean_mag`, `phot_rp_mean_mag`
 - Pan-Starss filters:`g_mean_psf_mag`,`r_mean_psf_mag`,`i_mean_psf_mag`,`z_mean_psf_mag`,`y_mean_psf_mag`
 - 2MASS filters: `j_m`, `h_m`, `ks_m`

You will also need to download the `PARSEC.model` file from the [BASE-9 models GitHub repo](https://github.com/BayesianStellarEvolution/base-models) (or another model set, if you are not using PARSEC, and edit the code below accordingly.)

In [ ]:
from bokeh.plotting import *
output_notebook()

In [ ]:
#interactive isochrone tool, may need to download additional dependencies to run

layout = cluster.createInteractiveIsochrone('PARSEC.model', 
                                            initialGuess = [
                                                cluster.yamlInputDict['logAge'][0], 
                                                cluster.yamlInputDict['Fe_H'][0],
                                                cluster.yamlInputDict['distMod'][0],
                                                cluster.yamlInputDict['Av'][0]
                                            ],
                                   mag ='phot_g_mean_mag', color1 ='phot_bp_mean_mag', color2 ='phot_rp_mean_mag', xrng = [-4,4], yrng = [24,5])
# in order to run a Python callback in a Jupyter notbook, you need to include the following
def bkapp(doc):
    doc.add_root(layout)
    
show(bkapp)

## (Optional) Interactively identify the region in parameter space containing the cluster members

This may be helpful for a cluster that is heavily embedded in a rich field when our automated algorithms fail.   For instance, tou could use this prior to exectuing the `cluster.runAll` function above.  After identifying helpful imits here to narrow down the cluster members, you could apply those limits the `cluster.data` and then attempt `cluster.runAll`.


In [ ]:
# in case you want to read data from a file
# cluster = GaiaClusterMembers()
# cluster.deredden = False
# cluster.pass_no = 2
# cluster.readDataFromFile('myGaiaData.csv')

layout = cluster.createInteractiveSelector()
def bkapp(doc):
    doc.add_root(layout)
show(bkapp)